# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = "../WeatherPy/output_data/cities.csv"
city_data=pd.read_csv(data_file)

city_data=city_data.iloc[:,1:10]
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,puerto ayora,-0.74,-90.35,80.60,61,20,16.11,EC,1592856963
1,butaritari,3.07,172.79,82.81,76,27,12.93,KI,1592856964
2,arraial do cabo,-22.97,-42.03,86.00,78,0,6.93,BR,1592856964
3,mar del plata,-38.00,-57.56,53.01,87,100,6.93,AR,1592856866
4,bluff,-46.60,168.33,39.00,83,1,3.00,NZ,1592856964


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_data[["Lat", "Lng"]].astype(float)

humidity = city_data['Humidity'].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_city=city_data.loc[ (city_data["Max Temp"].astype('float')<80) & (city_data["Max Temp"].astype('float')>70)
                        & (city_data["Wind Speed"].astype('float')<10) & (city_data["Cloudiness"].astype('float')==0)]
ideal_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,wewak,-3.55,143.63,79.21,80,0,3.67,PG,1592856967
85,tarragona,41.13,1.25,73.99,83,0,2.24,ES,1592856974
109,san pedro del pinatar,37.84,-0.79,75.20,69,0,8.05,ES,1592856977
161,marsa matruh,31.35,27.25,71.60,73,0,5.82,EG,1592856982
208,san vicente,-26.62,-54.13,74.32,62,0,3.29,AR,1592856987
242,mantua,45.17,10.78,78.01,58,0,2.55,IT,1592856991
243,popondetta,-8.75,148.25,72.12,93,0,2.57,PG,1592856991
314,uruzgan,32.83,66.00,74.30,18,0,2.19,AF,1592857001
327,luanda,-8.84,13.23,71.60,88,0,9.17,AO,1592857002
338,makokou,0.57,12.86,71.69,87,0,2.37,GA,1592857003


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=pd.DataFrame({'City':ideal_city['City'],'Country':ideal_city['Country']})
hotel_df['Lat']=""
hotel_df['Lng']=""
hotel_df['Hotel Name']=""
hotel_df['Hotel Address']=""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name,Hotel Address
37,wewak,PG,,,,
85,tarragona,ES,,,,
109,san pedro del pinatar,ES,,,,
161,marsa matruh,EG,,,,
208,san vicente,AR,,,,


In [6]:
params = {"key":g_key}

for index,row in hotel_df.iterrows():
    base_url="https://maps.googleapis.com/maps/api/geocode/json"
    
    city=row['City']
    params['address']=f"{city}"
    cities_lat_lng = requests.get(base_url, params=params)
    cities_lat_lng = cities_lat_lng.json()
    hotel_df.loc[index,"Lat"]=cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    hotel_df.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    
hotel_df.head()
    

,City,Country,Lat,Lng,Hotel Name,Hotel Address
37,wewak,PG,-3.58002,143.658,,
85,tarragona,ES,41.1189,1.24449,,
109,san pedro del pinatar,ES,37.8355,-0.791313,,
161,marsa matruh,EG,31.3543,27.2373,,
208,san vicente,AR,31.7235,-106.355,,


In [7]:
params = {
    "radius":5000,
    "types":"hotel",
    "keyword":"hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df = hotel_df.dropna(how='any')
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name,Hotel Address
37,wewak,PG,-3.58002,143.658,Sea View Hotel,Wewak
85,tarragona,ES,41.1189,1.24449,Hotel Lauria,"Rambla Nova, 20, Tarragona"
109,san pedro del pinatar,ES,37.8355,-0.791313,Thalasia Costa De Murcia,"Avenida del Puerto, 327-329, San Pedro del Pin..."
161,marsa matruh,EG,31.3543,27.2373,Beau Site,El Corniche Street Qesm
208,san vicente,AR,31.7235,-106.355,Ramada by Wyndham El Paso,"8250 Gateway Blvd E, El Paso"
242,mantua,IT,45.1564,10.7914,Best Western Hotel Cristallo,"Via Cisa, 1E, Cerese"
243,popondetta,PG,-8.7613,148.238,Comfort Inn,Popondetta
314,uruzgan,AF,32.9271,66.1415,,
327,luanda,AO,-8.81466,13.2302,EPIC SANA Luanda,"R. da Missão, Luanda"
338,makokou,GA,0.563317,12.8572,Hôtel Bélinga,Makokou


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# Add marker layer ontop of heat map
# Display figure
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))